### Station Data

In [1]:
import pandas as pd
import warnings
import jupyternotify
warnings.filterwarnings('ignore')
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [2]:
stations = pd.read_csv('station_availability_df', index_col=0).reset_index()
#only in service stations
station_availabilities = stations.loc[stations['in_service']==1,['dock_id','date','avail_bikes','avail_docks', 'tot_docks', '_lat','_long']]
station_availabilities['date'] = pd.to_datetime(station_availabilities['date'], format = '%Y-%m-%d %H:%M:%S')
station_availabilities.head(1)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long
0,72,2017-01-01,26,11,39,40.767272,-73.993929


### Bike Data

In [3]:
bike_trips = pd.read_csv('bike_trip_df',index_col = 0).reset_index(drop = True)

In [4]:
import datetime
bike_trips['starttime'] = pd.to_datetime(bike_trips['starttime'], format = '%Y-%m-%d %H:%M:%S')
bike_trips['stoptime'] = pd.to_datetime(bike_trips['stoptime'], format = '%Y-%m-%d %H:%M:%S')
#round all times to earliest hour
bike_trips['starttime'] = bike_trips['starttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day))
bike_trips['stoptime'] = bike_trips['stoptime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day))

#create two dataframes counting bike trips aggregated by timestamp and station id
outgoing_trips = bike_trips[['starttime','start station id']]
incoming_trips = bike_trips[['stoptime','end station id']]
incoming_bike_trips = incoming_trips.groupby(['stoptime','end station id']).size().reset_index()
outgoing_bike_trips = outgoing_trips.groupby(['starttime','start station id']).size().reset_index()

In [5]:
#merge two dataframes to find net bikes being added to the station
all_trips = pd.merge(incoming_bike_trips,outgoing_bike_trips, left_on = ['stoptime','end station id'], right_on = ['starttime','start station id'], how = 'outer', copy = False)
all_trips = all_trips.rename(index = str, columns = {"stoptime": "date", "end station id":"dock_id", "0_x":"incoming_bikes","0_y":"outgoing_bikes"})
all_trips['date'].fillna(all_trips['starttime'], inplace = True)
all_trips['dock_id'].fillna(all_trips['start station id'], inplace = True)
all_trips['incoming_bikes'].fillna(0, inplace = True)
all_trips['outgoing_bikes'].fillna(0, inplace = True)
all_trips.drop(columns = ['starttime','start station id'], inplace = True)

In [6]:
all_trips['net_bikes'] = all_trips.incoming_bikes-all_trips.outgoing_bikes
all_trips = all_trips.drop(columns = ['incoming_bikes','outgoing_bikes'])

In [7]:
all_trips.head()

,date,dock_id,net_bikes
0,2017-01-01,72.0,2.0
1,2017-01-01,79.0,-11.0
2,2017-01-01,82.0,1.0
3,2017-01-01,83.0,1.0
4,2017-01-01,116.0,-12.0


### Rebalancing Calculations

In [8]:
#combine station availability data to net incoming bike data
df = pd.merge(station_availabilities,all_trips, on = ['dock_id','date'], how = 'outer', copy = False).sort_values(by = ['dock_id', 'date'], ascending = True)
#replace all instances when no bikes left or arrived with 0
df.net_bikes.fillna(0,inplace=True)
df.dropna(inplace = True)
df.head(5)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long,net_bikes
0,72.0,2017-01-01,26.0,11.0,39.0,40.767272,-73.993929,2.0
1,72.0,2017-01-01,26.0,11.0,39.0,40.767272,-73.993929,2.0
2,72.0,2017-01-01,26.0,11.0,39.0,40.767272,-73.993929,2.0
3,72.0,2017-01-01,26.0,11.0,39.0,40.767272,-73.993929,2.0
4,72.0,2017-01-01,26.0,11.0,39.0,40.767272,-73.993929,2.0


In [9]:
df = df.drop_duplicates(subset=['dock_id','date'], keep='last').reset_index(drop=True)
df.head(5)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long,net_bikes
0,72.0,2017-01-01,21.0,16.0,39.0,40.767272,-73.993929,2.0
1,72.0,2017-01-02,32.0,5.0,39.0,40.767272,-73.993929,10.0
2,72.0,2017-01-03,37.0,2.0,39.0,40.767272,-73.993929,-3.0
3,72.0,2017-01-04,24.0,15.0,39.0,40.767272,-73.993929,-18.0
4,72.0,2017-01-05,6.0,30.0,39.0,40.767272,-73.993929,-4.0


In [10]:
#create column calculating how many bikes should be available at the next hour
df['theoretical_eod_avail_bikes'] = df.avail_bikes+df.net_bikes
df.head(100)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long,net_bikes,theoretical_eod_avail_bikes
0,72.0,2017-01-01,21.0,16.0,39.0,40.767272,-73.993929,2.0,23.0
1,72.0,2017-01-02,32.0,5.0,39.0,40.767272,-73.993929,10.0,42.0
2,72.0,2017-01-03,37.0,2.0,39.0,40.767272,-73.993929,-3.0,34.0
3,72.0,2017-01-04,24.0,15.0,39.0,40.767272,-73.993929,-18.0,6.0
4,72.0,2017-01-05,6.0,30.0,39.0,40.767272,-73.993929,-4.0,2.0
5,72.0,2017-01-06,4.0,35.0,39.0,40.767272,-73.993929,-2.0,2.0
6,72.0,2017-01-07,2.0,37.0,39.0,40.767272,-73.993929,-5.0,-3.0
7,72.0,2017-01-08,3.0,36.0,39.0,40.767272,-73.993929,2.0,5.0
8,72.0,2017-01-09,7.0,32.0,39.0,40.767272,-73.993929,-1.0,6.0
9,72.0,2017-01-10,12.0,27.0,39.0,40.767272,-73.993929,3.0,15.0


In [11]:
#shift inventory numbers back one hour to compare between how many bikes should be available 
#and how many bikes actually are
df[['actual_eod_avail_bikes','date+1']] = df.groupby(['dock_id'])['avail_bikes','date'].shift(-1)
df = df.drop(df[df.date+datetime.timedelta(days=1)!=df['date+1']].index).reset_index(drop=True).drop(columns ='date+1')
df.head(10000)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long,net_bikes,theoretical_eod_avail_bikes,actual_eod_avail_bikes
0,72.0,2017-01-01,21.0,16.0,39.0,40.767272,-73.993929,2.0,23.0,32.0
1,72.0,2017-01-02,32.0,5.0,39.0,40.767272,-73.993929,10.0,42.0,37.0
2,72.0,2017-01-03,37.0,2.0,39.0,40.767272,-73.993929,-3.0,34.0,24.0
3,72.0,2017-01-04,24.0,15.0,39.0,40.767272,-73.993929,-18.0,6.0,6.0
4,72.0,2017-01-05,6.0,30.0,39.0,40.767272,-73.993929,-4.0,2.0,4.0
5,72.0,2017-01-06,4.0,35.0,39.0,40.767272,-73.993929,-2.0,2.0,2.0
6,72.0,2017-01-07,2.0,37.0,39.0,40.767272,-73.993929,-5.0,-3.0,3.0
7,72.0,2017-01-08,3.0,36.0,39.0,40.767272,-73.993929,2.0,5.0,7.0
8,72.0,2017-01-09,7.0,32.0,39.0,40.767272,-73.993929,-1.0,6.0,12.0
9,72.0,2017-01-10,12.0,27.0,39.0,40.767272,-73.993929,3.0,15.0,11.0


In [12]:
df['bikes_added_by_citibike'] = df.actual_eod_avail_bikes - df.theoretical_eod_avail_bikes
df.head(5)

,dock_id,date,avail_bikes,avail_docks,tot_docks,_lat,_long,net_bikes,theoretical_eod_avail_bikes,actual_eod_avail_bikes,bikes_added_by_citibike
0,72.0,2017-01-01,21.0,16.0,39.0,40.767272,-73.993929,2.0,23.0,32.0,9.0
1,72.0,2017-01-02,32.0,5.0,39.0,40.767272,-73.993929,10.0,42.0,37.0,-5.0
2,72.0,2017-01-03,37.0,2.0,39.0,40.767272,-73.993929,-3.0,34.0,24.0,-10.0
3,72.0,2017-01-04,24.0,15.0,39.0,40.767272,-73.993929,-18.0,6.0,6.0,0.0
4,72.0,2017-01-05,6.0,30.0,39.0,40.767272,-73.993929,-4.0,2.0,4.0,2.0


### Labeling

In [13]:
### this is to get indexes of full or empty stations
#loop through each row
no_rebalancing_df = df.loc[:,['dock_id','date','avail_bikes','avail_docks', 'tot_docks', 'net_bikes','bikes_added_by_citibike', '_lat', '_long']].copy().sort_values(by=['dock_id','date'])

full_times = []
full_stations = []
empty_times = []
empty_stations = []
final_df = pd.DataFrame()
bookmark = 0

#how do i get the for loop to go back one step
for i in no_rebalancing_df.index:
    #if available bikes hits 0, save index and set the start point to that row index
    if no_rebalancing_df.iloc[i].avail_bikes<=0:
        final_df = final_df.append(no_rebalancing_df.iloc[bookmark:i+1,:])
        bookmark = i+1
        empty_stations.append(no_rebalancing_df.iloc[i].dock_id)
        empty_times.append(no_rebalancing_df.iloc[i].date)
        no_rebalancing_df = df.loc[:,['dock_id','date','avail_bikes','avail_docks', 'tot_docks', 'net_bikes','bikes_added_by_citibike', '_lat', '_long']].copy()
    #if available bikes hits 0, save index and set the start point to that row index
    elif no_rebalancing_df.iloc[i].avail_docks<=0:
        final_df = final_df.append(no_rebalancing_df.iloc[bookmark:i+1,:])
        bookmark = i+1
        full_stations.append(no_rebalancing_df.iloc[i].dock_id)
        full_times.append(no_rebalancing_df.iloc[i].date)
        no_rebalancing_df = df.loc[:,['dock_id','date','avail_bikes','avail_docks', 'tot_docks', 'net_bikes','bikes_added_by_citibike', '_lat', '_long']].copy()
    #if bikes_added_by_citibike is not 0
    elif no_rebalancing_df.bikes_added_by_citibike[i]!=0:
        #subtract that number from all avail_bikes and add that number to all avail_docks for that dock_id after that time
        #save the dock_id of the row we're looking at
        dock_id = no_rebalancing_df.iloc[i].dock_id
        rebalancing_int = no_rebalancing_df.iloc[i].bikes_added_by_citibike
        #find the last row with the same dock_id, add 1, and subtract rebalanced bike number from slice from index + 1 to last row
        no_rebalancing_df.loc[i+1:no_rebalancing_df[no_rebalancing_df.dock_id == dock_id].index.max() + 1,'avail_bikes'] = no_rebalancing_df[i+1:no_rebalancing_df[no_rebalancing_df.dock_id == dock_id].index.max() + 1].avail_bikes - rebalancing_int
        no_rebalancing_df.loc[i+1:no_rebalancing_df[no_rebalancing_df.dock_id == dock_id].index.max() + 1,'avail_docks'] = no_rebalancing_df[i+1:no_rebalancing_df[no_rebalancing_df.dock_id == dock_id].index.max() + 1].avail_docks + rebalancing_int

In [14]:
empty_df = pd.DataFrame({'date':empty_times,'dock_id':empty_stations, 'value' : 1})
full_df = pd.DataFrame({'date':full_times,'dock_id':full_stations, 'value' : 2})
empty_full_df = pd.concat([empty_df,full_df]).reset_index(drop=True)

In [15]:
empty_full_df.head()

,date,dock_id,value
0,2017-01-27,72.0,1
1,2017-02-21,72.0,1
2,2017-02-23,72.0,1
3,2017-02-28,72.0,1
4,2017-04-09,72.0,1


In [16]:
final_df2 = pd.merge(final_df,empty_full_df, on=['date','dock_id'], how = 'outer')

In [17]:
final_df2.value.fillna(0, inplace=True)

In [19]:
final_df2.head()

,dock_id,date,avail_bikes,avail_docks,tot_docks,net_bikes,bikes_added_by_citibike,_lat,_long,value
0,72.0,2017-01-01,21.0,16.0,39.0,2.0,9.0,40.767272,-73.993929,0.0
1,72.0,2017-01-02,23.0,14.0,39.0,10.0,-5.0,40.767272,-73.993929,0.0
2,72.0,2017-01-03,33.0,6.0,39.0,-3.0,-10.0,40.767272,-73.993929,0.0
3,72.0,2017-01-04,30.0,9.0,39.0,-18.0,0.0,40.767272,-73.993929,0.0
4,72.0,2017-01-05,12.0,24.0,39.0,-4.0,2.0,40.767272,-73.993929,0.0


In [21]:
final_df2.dropna(inplace = True)

In [22]:
%%notify
final_df2.to_csv('final_df.csv')

<IPython.core.display.Javascript object>